## Getting true payoffs per country

In [18]:
# load data frame with transactions
import pandas as pd
import numpy as np
import pickle
import networkx as nx
from pprint import pprint
from datetime import datetime
from datetime import date 
from etl import data

df = data.df
df_ac = df.groupby('ac_location').count().sort_values('ct_id', ascending=False)
df_mc = df.groupby('mc_location').count().sort_values('ct_id', ascending=False)
df_sorted

,ct_id,date,ac_location,name,parent_name,mc_name
mc_location,,,,,,
Netherlands,557,557,557,557,557,557
Germany,477,477,477,477,477,477
United Kingdom,402,402,402,402,402,402
Luxembourg,337,337,337,337,337,337
Austria,247,247,247,247,247,247
Italy,188,188,188,188,188,188
Sweden,146,146,146,146,146,146
France,141,141,141,141,141,141
Belgium,131,131,131,131,131,131


In [2]:
# extract transactions for a specific year and put them in a graph
start_date = datetime(2013, 1, 1)
end_date = datetime(2014, 1, 5)
G = data.df2graph(start_date=start_date, end_date=end_date)
G

In [3]:
# sum in/out edges for all nodes
total_out = data.sum_edges_for_nodes(method=G.out_edges)
total_out_dict = dict(zip(data.countries,total_out))

total_in = data.sum_edges_for_nodes(method=G.in_edges)
total_in_dict = dict(zip(data.countries,total_in))
import pprint
df2 = pd.DataFrame.from_dict(total_out_dict, orient = 'index', columns=['out'])
df3 = pd.DataFrame.from_dict(total_in_dict, orient = 'index', columns=['in'])
dff = pd.concat([df2, df3], axis = 1)
def test_balance(col1, col2):
    return sum(col1)==sum(col2)
print(test_balance(dff['out'],dff['in']))
dff['payoff'] = dff['in']-dff['out']
dff

True


,out,in,payoff
Germany,47,49,2
Luxembourg,18,38,20
Netherlands,54,64,10
Italy,18,24,6
United Kingdom,53,14,-39
France,16,12,-4
Austria,18,34,16
Sweden,18,7,-11
Finland,3,11,8
Cyprus,6,6,0


In [3]:
dff.index

Index(['Netherlands', 'Germany', 'Luxembourg', 'Italy', 'Sweden', 'Austria',
       'United Kingdom', 'Finland', 'France', 'Spain', 'Belgium', 'Denmark',
       'Norway', 'Czech Republic', 'Cyprus', 'Ireland', 'Estonia', 'Poland'],
      dtype='object')

In [19]:
# extract transactions for a specific year and put them in a graph
df_net = pd.DataFrame()
df_in = pd.DataFrame()
df_out = pd.DataFrame()

for year in range(2009, 2019):
    start_date = datetime(year, 1, 1)
    end_date = datetime(year+1, 1, 5)
    G = data.df2graph( start_date=start_date, end_date=end_date)
    #pprint(list(G.edges.data()))

    # sum in/out edges for all nodes
    total_out = data.sum_edges_for_nodes(G.out_edges)
    total_out_dict = dict(zip(data.countries,total_out))

    total_in = data.sum_edges_for_nodes(G.in_edges)
    total_in_dict = dict(zip(data.countries,total_in))
    import pprint
    df2 = pd.DataFrame.from_dict(total_out_dict, orient = 'index', columns=['out'])
    df3 = pd.DataFrame.from_dict(total_in_dict, orient = 'index', columns=['in'])
    dff = pd.concat([df2, df3], axis = 1)
    def test_balance(col1, col2):
        return sum(col1)==sum(col2)
    assert test_balance(dff['out'],dff['in'])
    dff['payoff'] = dff['in']-dff['out']
    
    df_net[year]=dff['payoff']
    df_in[year] = dff['in']
    df_out[year] = dff['out']
df_net.set_index()
df_net.to_csv('C:/Users/Kinga/OneDrive/thesis/data/cbm/payoffs.csv')

In [16]:
df_net.index # countries included in the analysis

Index(['Germany', 'Luxembourg', 'Netherlands', 'Italy', 'United Kingdom',
       'France', 'Austria', 'Sweden', 'Finland', 'Cyprus', 'Belgium',
       'Ireland', 'Denmark', 'Estonia', 'Norway', 'Czech Republic', 'Latvia',
       'Poland', 'Spain', 'Slovakia', 'Malta', 'Romania', 'Lithuania',
       'Croatia', 'Hungary', 'Bulgaria', 'Switzerland'],
      dtype='object')

In [13]:
df_net.head() # AC-MC

,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Germany,35,49,34,29,2,15,-8,-9,-18,37
Luxembourg,4,9,10,32,20,26,0,9,33,43
Netherlands,-27,-24,-13,-32,10,7,2,7,-9,-54
Italy,9,9,22,12,6,2,4,6,1,21
United Kingdom,-7,-8,-5,-26,-39,-12,0,-1,-3,-49


In [14]:
df_in.head() #AC

,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Germany,51,63,66,72,49,49,27,45,77,92
Luxembourg,21,28,33,53,38,49,22,48,89,94
Netherlands,19,13,26,36,64,49,29,50,53,32
Italy,18,24,41,31,24,19,28,28,17,27
United Kingdom,5,17,22,23,14,9,8,18,29,40


In [15]:
df_out.head() #MC

,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Germany,16,14,32,43,47,34,35,54,95,55
Luxembourg,17,19,23,21,18,23,22,39,56,51
Netherlands,46,37,39,68,54,42,27,43,62,86
Italy,9,15,19,19,18,17,24,22,16,6
United Kingdom,12,25,27,49,53,21,8,19,32,89


In [10]:
dff.values.tolist()

[[28, 11],
 [8, 5],
 [3, 2],
 [16, 18],
 [37, 44],
 [5, 4],
 [16, 26],
 [1, 0],
 [77, 91],
 [5, 1],
 [0, 0],
 [21, 27],
 [46, 34]]

In [16]:
a = dff['out'].values

In [15]:
b = dff['in'].values

In [15]:
y_diff = y_true - y_pred

In [22]:
import math
df_diff = pd.DataFrame(data=y_diff,index=df_true.index,columns=df_true.columns)

In [20]:
from sklearn.metrics import mean_squared_error
y1 = np.array([[1,1,3],[1,1,3]])
y2=np.array([[0,0,-15],[1,0,-20]])

y1 = y_pred
y2 = y_true
mean_squared_error(y1, y2)

1473.9888888888886